## Scraping of Retsinformation DK

In [4]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import re

In [23]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

HF_TOKEN = os.environ.get("HF_TOKEN")
from huggingface_hub import HfApi, HfFolder

HfFolder.save_token(HF_TOKEN)

## Step 1: Scrape all URLS of vejledninger

In [4]:
#| export

def extract_vejledninger_url(url_list):
    #Function to extract all unqiue URLs from 1 search page
    
    driver = webdriver.Chrome()
    driver.get(url_list)
    time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    soup = soup.find("div", class_="search-result-list")
    
    urls = set()  # Using a set to store URLs to avoid duplicates
    for div in soup.find_all('div', class_='document-entry'):
        url = div.get('about')
        if url:
            urls.add(url)
    
    driver.quit()
    
    return urls

def make_url_list(n=5):
    #Function to loop over N pages of search results and extract all URLs, and return them as a set
    #Base URL for search of vejledninger
    url_list = 'https://www.retsinformation.dk/documents?dt=180&h=false&page={page}&ps=100&r=30'
    
    #Url for adding the correct prefix to the URLs (complete address)
    url_prefix = 'https://www.retsinformation.dk'
    url_set = set()
    
    #At time of writing, there are 5 pages of results 
    for page_no in range(0, n):
        vejledninger = extract_vejledninger_url(url_list.format(page=page_no))
        complete_url = [url_prefix + url for url in vejledninger]
        url_set = url_set.union(complete_url)
        
    return list(url_set)

In [5]:
#| export
url_list = make_url_list()

In [7]:
url_list[0:5]

['https://www.retsinformation.dk/eli/retsinfo/2019/9607',
 'https://www.retsinformation.dk/eli/retsinfo/2019/9922',
 'https://www.retsinformation.dk/eli/retsinfo/2003/20353',
 'https://www.retsinformation.dk/eli/retsinfo/2021/9596',
 'https://www.retsinformation.dk/eli/retsinfo/2005/10377']

## Step 2: Scrape HTML content of vejledninger (bs object)

**OBS**
Nogle vejledninger har tilsynedeladende ikke title i "Titel2" format, men derimod bare "Titel' feks tilfældet for pulverlakering vejledning, andre har class TITLE, og andre igen har ingen men blot font size = 5.... Og et par har slet ingen formatering af overskriften i HTML. 

In [8]:
#| export

def scrape_content(urls):
    # Dictionary to store results
    driver = webdriver.Chrome()
    result_dict = {}

    for url in urls:
        
        driver.get(url)
        
        # Wait for the page to load completely
        time.sleep(2)

        # Get the page source and parse it with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")
        content_div = soup.find("div", class_="document-content")

        if content_div:
            if content_div.find("p", class_="Titel2"):
                title = content_div.find("p", class_="Titel2").get_text(strip=True).replace("\n", "")
            elif content_div.find("p", class_="Titel"):
                title = content_div.find("p", class_="Titel").get_text(strip=True).replace("\n", "")
            elif content_div.find("h1", class_="TITLE"):
                title = content_div.find("h1", class_="TITLE").get_text(strip=True).replace("\n", "")
            elif content_div.find("font", {"size": "5"}):
                title = content_div.find("font", {"size": "5"}).get_text(strip=True).replace("\n", "")
            else:
                title = str(url)
                
            result_dict[title] = content_div
        else:
            print("Content not found for URL:", url)

    # Close the WebDriver
    driver.quit()

    return result_dict

**Note, scraping mighte take several hours**

In [9]:
#| export
vejledninger_raw = scrape_content(url_list)

**For brief testing, take first 5 URLS and print**

In [12]:
#vejledninger_raw = scrape_content(url_list[0:5])
#vejledninger_raw.keys()

dict_keys(['Arbejde med flyveaske', 'Vejledning om regulering af satser fra 1. januar 2020 efter lov om arbejdsskadesikring, lov om sikring mod følger af arbejdsskade, lov om arbejdsskadeforsikring og lov om forsikring mod følger af ulykkestilfælde', 'Flugtveje   og sikkerhedsbelysning (nødbelysning) på faste arbejdssteder - At-vejledning   A.1.10 - December 2003 – Erstatter At-meddelelse nr. 1.01.2 af januar   1989', 'At-vejledning 13.0.1-1 om undervisningspligtige unges arbejde', 'Arbejdsrelateret muskel- og skeletbesvær - At-vejledning D.3.4 - Maj 2005'])

In [22]:
vejledninger_raw['Arbejde med flyveaske'].get_text(strip=True).replace("\n", "")[0:1000]

'Arbejde med flyveaskeSundhedsfarer og forebyggelse ved arbejde med flyveaske.At-vejledning D.2.21-11. februar - Opdateret juli 2019Erstatter At-meddelelse nr. 4.04.17 af oktober 1990Denne vejledning oplyser om de sundhedsfarer, der er forbundet med arbejde med flyveaske, og om, hvilke foranstaltninger der skal træffes for at imødegå dem.Flyveaske anvendes som fyldmateriale i forbindelse med vejbygning, i cement- og betonindustrien og ved produktion af gasbeton.SundhedsfareFlyveaske består af finkornede partikler, der udskilles af røggasserne fra kulfyrede kraftværker. Flyveaske er et variabelt produkt, hvis egenskaber og kemiske sammensætning afhænger af de anvendte kul, den anvendte forbrændingsteknik og røgrensningsteknikken.Flyveaske indeholder spormængder af bl.a. en række tungmetaller. Indholdet af krystallinsk siliciumdioxid, herunder α-kvarts, kan være over 0,1 pct., hvorfor unge under 18 år ikke må arbejde med flyveaske.Da asken er basisk, kan der ske irritation af hud og slim

## Step 3: Extract and clean text

In [19]:
#| export

def extract_text(bs_obj):
    # Find all <p> tags and get their text content
    paragraphs = bs_obj.find_all("p")
    
    # Concatenate the text content of each <p> tag, separated by a line break
    text = "\n".join(p.get_text(strip=True) for p in paragraphs)
    
    return text

def clean_text(text):
    # Replace two or more line breaks in a row with a double line break
    cleaned_text = re.sub(r'\n{2,}', '\n\n', text)
    return cleaned_text

def create_vejl_dict(vejledninger_raw):
    #vejledninger = scrape_content(urls)
    vejledninger_dict = {}
    for title, bs_obj in vejledninger_raw.items():
        text = extract_text(bs_obj)
        cleaned_text = clean_text(text)
        vejledninger_dict[title] = cleaned_text

In [20]:
#| export
vejledninger_text = create_vejl_dict(vejledninger_raw)

# Pushing to HF

( Loading previous CSV temporarily in order not to have to scrape again)

In [2]:
# Reload vejledninger_html_2-11-2023.csv into vejledninger_html_dict as key,value pairs
import csv
import sys

csv.field_size_limit(sys.maxsize)

vejledninger_tekst_dict = {}
csv_file_path = "data/data/vejledninger_tekst.csv"
with open(csv_file_path, "r", newline="", encoding="utf-8") as f:
    reader = csv.reader(f)
    # Skip first row
    next(reader)

    for row in reader:
        key = row[0]
        value = row[1]
        vejledninger_tekst_dict[key] = value

In [39]:
#Create hugging face dataset using the _from_dict method

from datasets import Dataset
vejledning_navn = list(vejledninger_tekst_dict.keys())
vejledning_tekst = list(vejledninger_tekst_dict.values())

ds_vejledning = Dataset.from_dict({"vejledning": vejledning_navn, "indhold": vejledning_tekst})

# add readme
ds_vejledning.info.description = """# Vejledninger fra Retsinformation.dk
Datasættet indeholder alle vejledninger scrapet fra Retsinformation.dk (pr. November 2023).
Datasættet indeholder 2 kolonner: navnet på den givne vejledning (såfremt det fremgik af html'en, ellers fremgår URL) og hele indholdet af vejledningen.
Teksten er renset og formateret således at der er 1 linjeskift (\n) mellem hver sektion ( <p> tag), 
medmindre der er indsat en eller flere tomme sektioner i træk hvormed der i stedet er indsat 2 linjeskift (\n\n)
"""

ds_vejledning.info.dataset_name = "Vejledninger fra Retsinformation.dk"
ds_vejledning.info.config_name = "data-retsinformation"

ds_vejledning.info

DatasetInfo(description="# Vejledninger fra Retsinformation.dk\nDatasættet indeholder alle vejledninger scrapet fra Retsinformation.dk (pr. November 2023).\nDatasættet indeholder 2 kolonner: navnet på den givne vejledning (såfremt det fremgik af html'en, ellers fremgår URL) og hele indholdet af vejledningen.\nTeksten er renset og formateret således at der er 1 linjeskift (\n) mellem hver sektion ( <p> tag), \nmedmindre der er indsat en eller flere tomme sektioner i træk hvormed der i stedet er indsat 2 linjeskift (\n\n)\n", citation='', homepage='', license='', features={'vejledning': Value(dtype='string', id=None), 'indhold': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, dataset_name='Vejledninger fra Retsinformation.dk', config_name='retsinformation', version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)

In [40]:
# upload dataset
ds_vejledning.push_to_hub(repo_id="dk_retrieval_benchmark", config_name="retsinformation")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]